In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install gymnasium pygame

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 5.0 MB/s eta 0:00:00


In [3]:
%pip install -U "gymnasium[classic-control]" "comet_ml>=3.44.0" stable-baselines3 "numpy<2.0.0"

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [4]:
from comet_ml.integration.gymnasium import CometLogger
from stable_baselines3 import A2C
import gymnasium as gym
import comet_ml
from comet_ml import Experiment, Optimizer


import numpy as np
import random
from collections import deque
from tensorflow.keras.layers import Input, Dense, Concatenate, Conv2D, Flatten,MaxPool2D
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras.losses import MeanSquaredError, MSE
from tensorflow import reduce_mean, convert_to_tensor, squeeze, float32, GradientTape
import gymnasium as gym
import numpy as np
import time
from PIL import Image
import os

In [5]:
class DQN:
    def __init__(self, state_space_shape, num_actions, model, target_model, learning_rate=0.1,
                 discount_factor=0.95, batch_size=16, memory_size=100):
        """
        Initializes Deep Q Network agent.
        :param state_space_shape: shape of the observation space
        :param num_actions: number of actions
        :param model: Keras model
        :param target_model: Keras model
        :param learning_rate: learning rate
        :param discount_factor: discount factor
        :param batch_size: batch size
        :param memory_size: maximum size of the experience replay memory
        """
        self.state_space_shape = state_space_shape
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)
        self.model = model
        self.target_model = target_model
        self.update_target_model()

    def update_memory(self, state, action, reward, next_state, done):
        """
        Adds experience tuple to experience replay memory.
        :param state: current state
        :param action: performed action
        :param reward: reward received for performing action
        :param next_state: next state
        :param done: if episode has terminated after performing the action in the current state
        """
        self.memory.append((state, action, reward, next_state, done))

    def update_target_model(self):
        """
        Synchronize the target model with the main model.
        """
        self.target_model.set_weights(self.model.get_weights())

    def get_action(self, state, epsilon):
        """
        Returns the best action following epsilon greedy policy for the current state.
        :param state: current state
        :param epsilon: exploration rate
        :return:
        """
        probability = np.random.random() + epsilon / self.num_actions
        if probability < epsilon:
            return np.random.randint(0, self.num_actions)
        else:
            if isinstance(self.state_space_shape, tuple):
                state = state.reshape((1,) + self.state_space_shape)
            else:
                state = state.reshape(1, self.state_space_shape)
            return np.argmax(self.model.predict(state,verbose=0)[0])

    def load(self, model_name):
        path="/content/drive/MyDrive/Colab Notebooks/abs_206009"
        """
        Loads the weights of the model at specified episode checkpoint.
        :param model_name: name of the model
        :param episode: episode checkpoint
        """
        self.model.load_weights(f'{path}/weights/{model_name}')

    def save(self, model_name, episode):
        """
        Stores the weights of the model at specified episode checkpoint.
        :param model_name: name of the model
        :param episode: episode checkpoint
        """
        path="/content/drive/MyDrive/Colab Notebooks/abs_206009"
        if not os.path.exists(f"{path}/weights/"):
            os.mkdir(f"{path}/weights")
        self.model.save_weights(f'{path}/weights/{model_name}_{episode}.h5')

    def train(self):
        """
        Performs one step of model training.
        """
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        if isinstance(self.state_space_shape, tuple):
            states = np.zeros((batch_size,) + self.state_space_shape)
        else:
            states = np.zeros((batch_size, self.state_space_shape))
        actions = np.zeros((batch_size, self.num_actions))

        for i in range(len(minibatch)):
            state, action, reward, next_state, done = minibatch[i]
            if done:
                max_future_q = reward
            else:
                if isinstance(self.state_space_shape, tuple):
                    next_state = next_state.reshape((1,) + self.state_space_shape)
                else:
                    next_state = next_state.reshape(1, self.state_space_shape)
                max_future_q = (reward + self.discount_factor *
                                np.amax(self.target_model.predict(next_state,verbose=0)[0]))
            if isinstance(self.state_space_shape, tuple):
                state = state.reshape((1,) + self.state_space_shape)
            else:
                state = state.reshape(1, self.state_space_shape)
            target_q = self.model.predict(state,verbose=0)[0]
            target_q[action] = max_future_q
            states[i] = state
            actions[i] = target_q

        self.model.train_on_batch(states, actions)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:
class DDQN:
    def __init__(self, state_space_shape, num_actions, model, target_model, learning_rate=0.1,
                 discount_factor=0.95, batch_size=16, memory_size=100):
        """
        Initializes Double Deep Q Network agent.
        :param state_space_shape: shape of the observation space
        :param num_actions: number of actions
        :param model: Keras model
        :param target_model: Keras model
        :param learning_rate: learning rate
        :param discount_factor: discount factor
        :param batch_size: batch size
        :param memory_size: maximum size of the experience replay memory
        """
        self.state_space_shape = state_space_shape
        self.num_actions = num_actions
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.batch_size = batch_size
        self.memory = deque(maxlen=memory_size)
        self.model = model
        self.target_model = target_model
        self.update_target_model()

    def update_memory(self, state, action, reward, next_state, done):
        """
        Adds experience tuple to experience replay memory.
        :param state: current state
        :param action: performed action
        :param reward: reward received for performing action
        :param next_state: next state
        :param done: if episode has terminated after performing the action in the current state
        """
        self.memory.append((state, action, reward, next_state, done))

    def update_target_model(self):
        """
        Synchronize the target model with the main model.
        """
        self.target_model.set_weights(self.model.get_weights())

    def get_action(self, state, epsilon):
        """
        Returns the best action following epsilon greedy policy for the current state.
        :param state: current state
        :param epsilon: exploration rate
        :return:
        """
        probability = np.random.random() + epsilon / self.num_actions
        if probability < epsilon:
            return np.random.randint(0, self.num_actions)
        else:
            if isinstance(self.state_space_shape, tuple):
                state = state.reshape((1,) + self.state_space_shape)
            else:
                state = state.reshape(1, self.state_space_shape)
            return np.argmax(self.model.predict(state,verbose=0)[0])

    def load(self, model_name):
        path="/content/drive/MyDrive/Colab Notebooks/abs_206009"
        """
        Loads the weights of the model at specified episode checkpoint.
        :param model_name: name of the model
        :param episode: episode checkpoint
        """
        self.model.load_weights(f'{path}/weights/{model_name}')

    def save(self, model_name, episode):
        """
        Stores the weights of the model at specified episode checkpoint.
        :param model_name: name of the model
        :param episode: episode checkpoint
        """
        path="/content/drive/MyDrive/Colab Notebooks/abs_206009"
        if not os.path.exists(f"{path}/weights/"):
            os.mkdir(f"{path}/weights")
        self.model.save_weights(f'{path}/weights/{model_name}_{episode}.h5')

    def train(self):
        """
        Performs one step of model training.
        """
        batch_size = min(self.batch_size, len(self.memory))
        minibatch = random.sample(self.memory, batch_size)

        if isinstance(self.state_space_shape, tuple):
            states = np.zeros((batch_size,) + self.state_space_shape)
        else:
            states = np.zeros((batch_size, self.state_space_shape))
        actions = np.zeros((batch_size, self.num_actions))

        for i in range(len(minibatch)):
            state, action, reward, next_state, done = minibatch[i]
            if done:
                max_future_q = reward
            else:
                if isinstance(self.state_space_shape, tuple):
                    next_state = next_state.reshape((1,) + self.state_space_shape)
                else:
                    next_state = next_state.reshape(1, self.state_space_shape)
                max_action = np.argmax(self.model.predict(next_state,verbose=0)[0])
                max_future_q = (reward + self.discount_factor *
                                self.target_model.predict(next_state,verbose=0)[0][max_action])
            if isinstance(self.state_space_shape, tuple):
                state = state.reshape((1,) + self.state_space_shape)
            else:
                state = state.reshape(1, self.state_space_shape)
            target_q = self.model.predict(state,verbose=0)[0]
            target_q[action] = max_future_q
            states[i] = state
            actions[i] = target_q

        self.model.train_on_batch(states, actions)

In [6]:
def build_model3(state_space_shape,num_actions):

    model = Sequential()

    model.add(Dense(state_space_shape[0], input_shape=state_space_shape,activation="relu"))
    model.add(Dense(16, activation="relu"))
    model.add(Dense(8, activation="relu"))
    model.add(Dense(num_actions, activation="linear"))

    model.compile(loss="mse", optimizer=Adam(learning_rate=0.0001))
    return model

In [7]:
api_key="Woe9tANOMbC6V0eSA75H6R4yR"
workspace="michael132"

In [ ]:
api_key=input("api_key:")
workspace=input("workspace:")

In [8]:
experiment = Experiment(project_name="ddqn-agent",api_key=api_key,workspace=workspace)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/michael132/ddqn-agent/bc55d7deaa724841b1697ce7dbe20cc9



In [9]:
hyperparameters = {
    "algorithm": "bayes",
    "parameters": {
        "learning_rate": {"type": "float", "min": 0.0001, "max": 0.01},
        "batch_size": {"type": "integer", "min": 16, "max": 64},
        "discount_factor": {"type": "float", "min": 0.85, "max": 0.99},
        "epsilon": {"type": "float", "min": 0.01, "max": 0.3}
    },
    "spec": {
        "metric": "reward",
        "goal": "maximize",
        "maxCombo": 10
    }
}

# Initialize the optimizer
optimizer = Optimizer(hyperparameters, api_key=api_key)


COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: 46881999de4e4311bd436d2456b2a89c
COMET INFO: Using optimizer config: {'algorithm': 'bayes', 'configSpaceSize': 'infinite', 'endTime': None, 'id': '46881999de4e4311bd436d2456b2a89c', 'lastUpdateTime': None, 'maxCombo': 10, 'name': '46881999de4e4311bd436d2456b2a89c', 'parameters': {'batch_size': {'max': 64, 'min': 16, 'scalingType': 'uniform', 'type': 'integer'}, 'discount_factor': {'max': 0.99, 'min': 0.85, 'scalingType': 'uniform', 'type': 'float'}, 'epsilon': {'max': 0.3, 'min': 0.01, 'scalingType': 'uniform', 'type': 'float'}, 'learning_rate': {'max': 0.01, 'min': 0.0001, 'scalingType': 'uniform', 'type': 'float'}}, 'predictor': None, 'spec': {'goal': 'maximize', 'gridSize': 10, 'maxCombo': 10, 'metric': 'reward', 'minSampleSize': 100, 'objective': 'minimize', 'retryAssignLimit': 0, 'retryLimit': 1000}, 'startTime': 152493184

In [10]:
env = gym.make("Acrobot-v1")

state_space_shape = env.observation_space.shape
num_actions = env.action_space.n

# # Uncomment if you want to Upload Videos of your environment to Comet
# # env = gym.wrappers.RecordVideo(env, 'test')

# experiment = comet_ml.Experiment()

# env = CometLogger(env, experiment)

# env.close()
# experiment.end()

In [ ]:
for experiment in optimizer.get_experiments(project_name='ddqn-acrobot'):
    # params = experiment.get_parameters()

    # Use parameters from Comet's optimizer
    # learning_rate = params["learning_rate"]
    # batch_size = params["batch_size"]
    # discount_factor = params["discount_factor"]
    # epsilon = params["epsilon"]
    # memory_size = params["memory_size"]

    learning_rate = experiment.get_parameter("learning_rate")
    batch_size = experiment.get_parameter("batch_size")
    discount_factor = experiment.get_parameter("discount_factor")
    epsilon = experiment.get_parameter("epsilon")


    # Rebuild model with new parameters
    model = build_model3(state_space_shape, num_actions)
    target_model = build_model3(state_space_shape, num_actions)
    agent = DDQN(state_space_shape, num_actions, model, target_model,
                 learning_rate=learning_rate, discount_factor=discount_factor, batch_size=batch_size, memory_size=1000000)

    # Training loop
    total_rewards = 0
    num_episodes = 100
    for episode in range(num_episodes):
        state, _ = env.reset()
        steps = 0
        terminated = False
        while not terminated:
            steps+=1
            action = agent.get_action(state, epsilon)
            new_state, reward, terminated, _, _ = env.step(action)

            if terminated:
                reward = int(10_000 / steps)

            total_rewards += reward
            agent.update_memory(state, action, reward, new_state, terminated)
            agent.train()
            state = new_state

    # Log results to Comet
    experiment.log_metric("reward", total_rewards / num_episodes)


COMET INFO: Optimizer metrics is 'reward' but no logged values found. Experiment ignored in sweep.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : mean_date_1974
COMET INFO:     url                   : https://www.comet.com/michael132/ddqn-acrobot/bf38f436580146cb9553285360446481
COMET INFO:   Others:
COMET INFO:     optimizer_count        : 1
COMET INFO:     optimizer_id           : b38836a25f8b4e39adca99cf4aa5befa
COMET INFO:     optimizer_metric       : reward
COMET INFO:     optimizer_metric_value : None
COMET INFO:     optimizer_name         : b38836a25f8b4e39adca99cf4aa5befa
COMET INFO:     optimizer_objective    : minimum
COMET INFO:     optimizer_parameters   : {"batch_size": 24, "discount

KeyboardInterrupt: 

In [11]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : integral_yak_6135
COMET INFO:     url                   : https://www.comet.com/michael132/ddqn-agent/bc55d7deaa724841b1697ce7dbe20cc9
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 


In [12]:
model = build_model3(state_space_shape, num_actions)
target_model = build_model3(state_space_shape, num_actions)
agent = DDQN(state_space_shape, num_actions, model, target_model,memory_size=1000000)

epsilon=0.143
num_episodes=50
for episode in range(1, num_episodes + 1):
    rewards = 0
    steps = 0
    state, _ = env.reset()
    terminated = False
    while not terminated:
        steps += 1
        action = agent.get_action(state, epsilon)
        new_state, reward, terminated, _, _ = env.step(action)

        if terminated:
            reward=int(10_000/steps)

        rewards += reward

        agent.update_memory(state, action, reward, new_state, terminated)
        state = new_state

    print(f"Episode {episode}/{num_episodes}, reward: {rewards}")

    agent.train()
    if episode % 60 == 0:
      epsilon *= 0.882
      epsilon=max(epsilon,0.001)

    if episode % 10 == 0:
        agent.update_target_model()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode 1/50, reward: -1093.0
Episode 2/50, reward: -2855.0
Episode 3/50, reward: -1285.0
Episode 4/50, reward: -720.0
Episode 5/50, reward: -3076.0


Episode 6/50, reward: -1461.0


Episode 7/50, reward: -1533.0
Episode 8/50, reward: -908.0
Episode 9/50, reward: -1081.0
Episode 10/50, reward: -1839.0
Episode 11/50, reward: -766.0
Episode 12/50, reward: -1002.0
Episode 13/50, reward: -2361.0
Episode 14/50, reward: -2081.0
Episode 15/50, reward: -711.0
Episode 16/50, reward: -2574.0
Episode 17/50, reward: -2061.0
Episode 18/50, reward: -812.0
Episode 19/50, reward: -1305.0
Episode 20/50, reward: -594.0
Episode 21/50, reward: -1268.0
Episode 22/50, reward: -1093.0
Episode 23/50, reward: -1325.0
Episode 24/50, reward: -738.0
Episode 25/50, reward: -1456.0
Episode 26/50, reward: -1398.0
Episode 27/50, reward: -791.0
Episode 28/50, reward: -1123.0
Episode 29/50, reward: -386.0
Episode 30/50, reward: -637.0
Episode 31/50, reward: -614.0
Episode 32/50, reward: -1183.0
Episode 33/50, reward: -638.0
Episode 34/50, reward: -742.0
Episode 35/50, reward: -1576.0
Episode 36/50, reward: -639.0
Episode 37/50, reward: -998.0
Episode 38/50, reward: -878.0
Episode 39/50, reward: -78

In [13]:
experiment = Experiment(project_name="dqn-agent",api_key=api_key,workspace=workspace)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/michael132/dqn-agent/4a0b363aa51440b88c97942e8e6df343

COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.


In [ ]:
for experiment in optimizer.get_experiments(project_name='ddqn-acrobot'):
    # params = experiment.get_parameters()

    # Use parameters from Comet's optimizer
    # learning_rate = params["learning_rate"]
    # batch_size = params["batch_size"]
    # discount_factor = params["discount_factor"]
    # epsilon = params["epsilon"]
    # memory_size = params["memory_size"]

    learning_rate = experiment.get_parameter("learning_rate")
    batch_size = experiment.get_parameter("batch_size")
    discount_factor = experiment.get_parameter("discount_factor")
    epsilon = experiment.get_parameter("epsilon")


    # Rebuild model with new parameters
    model = build_model3(state_space_shape, num_actions)
    target_model = build_model3(state_space_shape, num_actions)
    agent = DQN(state_space_shape, num_actions, model, target_model,
                 learning_rate=learning_rate, discount_factor=discount_factor, batch_size=batch_size, memory_size=1000000)

    # Training loop
    total_rewards = 0
    num_episodes = 100
    for episode in range(num_episodes):
        state, _ = env.reset()
        steps = 0
        terminated = False
        while not terminated:
            steps+=1
            action = agent.get_action(state, epsilon)
            new_state, reward, terminated, _, _ = env.step(action)

            if terminated:
                reward = int(10_000 / steps)

            total_rewards += reward
            agent.update_memory(state, action, reward, new_state, terminated)
            agent.train()
            state = new_state

    # Log results to Comet
    experiment.log_metric("reward", total_rewards / num_episodes)


COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : proper_cardinal_5829
COMET INFO:     url                   : https://www.comet.com/michael132/ddqn-agent/bc774ab0c3224752a92ca4f60f5170b6
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/micha

KeyboardInterrupt: 

In [14]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : profound_lease_4959
COMET INFO:     url                   : https://www.comet.com/michael132/dqn-agent/4a0b363aa51440b88c97942e8e6df343
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
